# Sensorfusion - Autonomes Fahren


In [189]:
# Imports
import pandas as pd
import numpy as np
from pyds import MassFunction
import time as time

In [190]:
data1=pd.read_csv("DPE16_1.csv", sep=';', decimal=",")
data2=pd.read_csv("DPE16_2.csv", sep=';', decimal=",")
data3=pd.read_csv("DPE16_3.csv", sep=';', decimal=",")
data4=pd.read_csv("DPE16_4.csv", sep=';', decimal=",")


#TODO rename column Bbox_Hoehe

In [191]:
data1

,t,Eigengeschwindigkeit(m/s),Bbox_Breite,Bbox_Hoehe,Bbox_links_unten_y,Abstand(m)
0,0.0,22.0,90,99,153,76.5
1,0.1,22.2,90,100,154,77.0
2,0.2,22.2,86,95,154,77.0
3,0.3,22.1,90,100,153,76.5
4,0.4,22.1,90,100,153,76.5
...,...,...,...,...,...,...
99,9.9,22.1,154,171,117,58.5
100,10.0,22.1,157,174,114,57.0
101,10.1,22.0,158,175,116,58.0
102,10.2,22.2,157,174,114,57.0


# Notizen
Jede Zeile entspricht Messung.
Verhältnis Breite/Höhe durch BBox-Werte.
Geschwindigkeit nur über Zeit (min Anzahl an Messwerten?) durch Abstand Differenz - Eigengeschwindigkeit
Beschleunigung über Zeit (sobald 2 Werte für Geschwindigkeit vorhanden: GeschwindigkeitNeu - GeschwindigkeitAlt

Konfidenz höher, je kleiner der Abstand
Omega: 1 - Konfidenz
omega: P L M F



In [192]:

#Save speed of previous measurement
oldSpeedSaved = None
newSpeedSaved = None


In [193]:
# Higher value means bycicle or motorbike (1.6 - 2.4)
# lower value means car (0.4 - 1.0)
# not in the task but we think equal ratios should increase the probability for truck
# values around 1 mean truck (0.8 - 1.6)
def calcBoundingRatio(height, width):
    ratio = height / width
    
    return ratio
    
    
# Manage saved values of speed to calculate acceleration
# Only called by calcSpeed
def saveSpeed(speed):
    global newSpeedSaved
    global oldSpeedSaved
    if (newSpeedSaved):
        oldSpeedSaved = newSpeedSaved
    newSpeedSaved = speed
    
#calculate speed
def calcSpeed(oldOwnSpeed, newOwnSpeed, oldDistance, newDistance):
    global newSpeedSaved
    # Check if there are null values for distance
    if (pd.isna(newDistance)):
        print('New Distance is NaN')
        print('Do not evaluate speed due to measurement error')
        saveSpeed(None)
        
        return
    
    difDistance = newDistance - oldDistance
    # Use average speed because old and new speed can be slightly different
    # Sensors are not accurate enough. They can only measure in 0,5m steps
    # Therefore a lot of weird speed data is calculated
    ownSpeed = (oldOwnSpeed + newOwnSpeed) / 2
    speed = difDistance / 0.1 + ownSpeed
    saveSpeed(speed)
    
    return speed
    
# calculate acceleration with formula: dif speed / dif time
def calcAcceleration():
    global newSpeedSaved
    global oldSpeedSaved
    #Catch oldSpeed null for first measurement
    if ((oldSpeedSaved == None) | (newSpeedSaved == None)):
        print('Keine Daten zur Beschleunigung vorhanden')
        # Return 0 by guessing, two measurements are required.
        # No problem because acceleration will only have effect if it is high
        return 0
    speedDif = newSpeedSaved - oldSpeedSaved
    acc = speedDif / 0.1
    
    return acc
    
#def calcMasses(oldMeasurement, newMeasurement):
    # Call other calcs
    
testspd1 = calcSpeed(22.2, 22.1, 77.0, 76.5)
print(testspd1)
testspd2 = calcSpeed(22.1, 22.1, 76.5, 76.5)
print(testspd2)
testspd3 = calcSpeed(22.1, 22.1, 76.5, np.nan)
#testspd4 = calcSpeed(22.1, 22.1, 76.5, 76.5)



testacc = calcAcceleration()
print(testacc)

17.15
22.1
New Distance is NaN
Do not evaluate speed due to measurement error
Keine Daten zur Beschleunigung vorhanden
0


In [194]:
omega = 'plmf'

def probabilityFromRatio(ratio, distance):
    
    omeg = 0.2
    p = l = m = f = 0

    if ((ratio > 1.6) & (ratio < 2.4)):
        m = f = (1 - omeg) / 2

    if ((ratio < 1.6) & (ratio > 0.4)):
        p = l = (1 - omeg) / 2
     
    mass = MassFunction({'p': p, 'l': l, 'm': m, 'f': f, omega: omeg})   
    return mass

# Speed:
# PKW, LKW, Motorrad: 50km/h - 90km/h ==> ~13.89m/s - 25m/s
def probabilityFromSpeed(speed):

    omeg = 0.2

    # Probability of Speed is equal because of speed limit
    # Task requirement
    # p = l = m = f = 0.0
    
    p = l = m = f = 0

    if ((speed > 13.88) & (speed < 25)):
        m = p = l = (1 - omeg) / 3

    mass = MassFunction({'p': p, 'l': l, 'm': m, 'f': f, omega: omeg})
    return mass

def probabilityFromAcc(acc):

    omeg = 0.2

    p = l = m = f = 0

    # Task requirement
    if (acc > 1000):
        m = (1 - omeg)

    mass = MassFunction({'p': p, 'l': l, 'm': m, 'f': f, omega: omeg})
    return mass

In [195]:
def iterateMeasurement(data):
    
    massResult = None

    #for row in data.items():
    for index, row in data.iterrows():

        # Data Print
        # print('---------------------------------------------------')
        # print('Zeitpunkt:', row['t'])
        # Evaluate Ratio
        ratio = calcBoundingRatio(row['Bbox_Hoehe'], row['Bbox_Breite'])
        # print('Ratio:', ratio)

        # TODO: Use MassFunction? to evaluate the result
        
        
        #def calcSpeed(oldOwnSpeed, newOwnSpeed, oldDistance, newDistance):
        if(index==0):
            continue
            
        speed = calcSpeed(data.loc[(index - 1),'Eigengeschwindigkeit(m/s)'],
                            row['Eigengeschwindigkeit(m/s)'],
                            data.loc[(index - 1),'Abstand(m)'],
                            row['Abstand(m)'])
        # print('Geschwindigkeit: ', speed)
        
        acc = calcAcceleration()
        # print('Acceleration: ', acc)
        
        # Simulate 100 ms clock
        # Uncomment for faster execution
        #time.sleep(0.1)

        # MassFunction 
        massR = probabilityFromRatio(ratio, 0)
        # print('MassR: ', massR) 
        massS = probabilityFromSpeed(speed)
        # print('MassS: ', massS) 
        massA = probabilityFromAcc(acc)
        # print('MassA: ', massA)

        mass = massR.combine_conjunctive(massS).combine_conjunctive(massA)
        # print('Mass: ', mass)
        
        if (massResult is None):
            massResult = mass
        else:
            massResult = massResult.combine_conjunctive(mass)
            
    print('MassResult: ', massResult)



iterateMeasurement(data3)
#iterateMeasurement(data2)
#iterateMeasurement(data3)
#iterateMeasurement(data4)

MassResult:  {{'p'}:0.5; {'l'}:0.5; {'m'}:3.5931948471895447e-50; {'m', 'p', 'l', 'f'}:4.745623532236021e-63; {'f'}:0.0}
